gm_sandbox.ipynb

In [18]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import requests
import time # use for scraping code lag

In [19]:
import requests
import pandas as pd

def get_nba_team_stats():
    """
    Get NBA team stats for second half of games and return as a DataFrame.
    """
    url = "https://stats.nba.com/stats/leaguedashteamstats"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
        'Referer': 'https://www.nba.com/',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Origin': 'https://www.nba.com'
    }
    
    params = {
        'MeasureType': 'Base',
        'PerMode': 'PerGame',
        'PlusMinus': 'N',
        'PaceAdjust': 'N',
        'Rank': 'N',
        # 'Season': '2023-24',
        'Season': '2024-25',
        'SeasonType': 'Regular Season',
        'GameSegment': 'Second Half',
        'Period': '0',
        'LastNGames': '0',
        'Month': '0',
        'OpponentTeamID': '0',
        'PORound': '0',
        'SeasonSegment': '',
        'DateFrom': '',
        'DateTo': ''
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Raise exception for bad status codes
        
        data = response.json()
        headers = data['resultSets'][0]['headers']
        rows = data['resultSets'][0]['rowSet']
        
        # Create DataFrame
        df = pd.DataFrame(rows, columns=headers)
        
        # Select and rename columns to match the website display
        columns_mapping = {
            'TEAM_NAME': 'Team',
            'GP': 'GP',
            'W': 'W',
            'L': 'L',
            'W_PCT': 'WIN%',
            'MIN': 'MIN',
            'PTS': 'PTS',
            'FGM': 'FGM',
            'FGA': 'FGA',
            'FG_PCT': 'FG%',
            'FG3M': '3PM',
            'FG3A': '3PA',
            'FG3_PCT': '3P%',
            'FTM': 'FTM',
            'FTA': 'FTA',
            'FT_PCT': 'FT%',
            'OREB': 'OREB',
            'DREB': 'DREB',
            'REB': 'REB',
            'AST': 'AST',
            'TOV': 'TOV',
            'STL': 'STL',
            'BLK': 'BLK',
            'BLKA': 'BLKA',
            'PF': 'PF',
            'PFD': 'PFD',
            'PLUS_MINUS': '+/-'
        }
        
        # Select and rename columns
        df = df[list(columns_mapping.keys())].rename(columns=columns_mapping)
        
        # Sort by WIN% descending
        df = df.sort_values('WIN%', ascending=False).reset_index(drop=True)
        
        # Format percentages
        for col in ['FG%', '3P%', 'FT%']:
            df[col] = (df[col] * 100).round(1)
        
        return df
        
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [20]:
# execute
nba_df = get_nba_team_stats()

print(len(nba_df))
print("\nNBA Team Stats (Second Half):")
display(nba_df.head(20))

30

NBA Team Stats (Second Half):


,Team,GP,W,L,WIN%,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-
0,Cleveland Cavaliers,31,27,4,0.871,24.0,59.6,21.7,43.7,49.7,7.3,19.0,38.3,8.9,11.7,76.0,5.4,16.2,21.6,13.5,6.8,4.5,2.3,2.6,10.0,11.1,3.5
1,Oklahoma City Thunder,31,26,5,0.839,24.0,55.6,20.5,45.5,45.1,5.7,18.4,30.8,8.9,10.7,82.6,5.9,16.5,22.4,12.1,5.7,5.6,3.3,3.0,11.1,9.9,4.4
2,Boston Celtics,32,23,9,0.719,24.0,57.3,19.9,44.1,45.2,8.3,23.8,35.0,9.1,11.6,78.6,5.8,16.3,22.1,11.7,6.1,3.6,2.9,2.1,9.1,10.1,3.5
3,New York Knicks,32,22,10,0.688,24.0,58.0,21.2,42.6,49.7,6.8,17.5,39.1,8.8,10.7,81.6,4.6,15.7,20.2,13.4,6.6,3.8,2.3,2.3,9.8,9.9,3.0
4,Memphis Grizzlies,33,22,11,0.667,24.0,60.3,21.4,44.8,47.7,6.4,18.2,35.4,11.1,14.2,78.4,6.2,16.9,23.1,14.2,9.0,4.5,3.4,2.6,11.8,11.9,1.8
5,Houston Rockets,32,21,11,0.656,24.0,56.4,20.4,45.8,44.4,5.5,18.3,30.3,10.2,13.0,78.3,7.2,16.8,24.0,11.0,6.8,4.8,2.8,2.8,11.0,10.8,2.0
6,Dallas Mavericks,32,20,12,0.625,24.0,59.0,21.3,44.1,48.3,6.7,17.7,37.8,9.7,12.3,79.1,5.6,15.9,21.5,12.2,6.5,4.0,2.8,2.2,9.7,9.9,2.2
7,Orlando Magic,34,20,14,0.588,24.0,53.5,18.5,41.4,44.7,6.1,18.8,32.2,10.4,13.3,78.1,5.3,16.6,21.9,11.4,7.3,4.6,3.1,1.8,11.0,11.5,5.6
8,LA Clippers,31,18,13,0.581,24.0,55.7,20.2,41.7,48.5,6.3,16.3,38.5,9.0,11.5,77.9,5.1,16.2,21.3,11.7,8.4,4.9,2.2,2.4,10.1,9.7,1.6
9,Los Angeles Lakers,31,18,13,0.581,24.0,53.8,18.9,41.7,45.4,5.9,17.4,33.9,10.1,13.0,77.2,4.5,15.4,19.9,12.7,6.4,3.4,2.1,2.2,8.8,10.9,-3.3
